In [1]:
# 处理测试集 xlsx中的脏数据

import openpyxl
import jieba
import re

# 加载 哈工大的停用词表 清洗用到
with open("../data/hit_stopwords.txt", 'r', encoding='utf-8') as f:
    stopword_list = [w.strip('\n') for w in f.readlines()]

filePath = "../data/type.xlsx"
wb = openpyxl.load_workbook(filePath)   # 打开excel文件
ws = wb['类别']     # 定位 sheet
results = []

# 这里把title 和 conten结合成一个str
for i in range(2, ws.max_row+1):
    lists = []
    strs = ""
    strs = strs + ws.cell(i, 3).value + " " + ws.cell(i, 4).value

    # 去除中文标点符号
    strs = re.sub("[\s+\.\!\-\/_,$%^*()+\"\']+|[+——！，。？、~@#￥%……&*（）：”“]", "",strs)
    # 使用jieba进行分词，将文本分成词语列表
    words = jieba.lcut(strs)
    result = ""
    # 然后清除停用词语
    for word in words:
        if word not in stopword_list:
            result = result + word + " "


    lists.append(ws.cell(i, 1).value)
    lists.append(result)
    results.append(lists)

# 得到一个list 然后写入csv
# print(results)

In [2]:
# 生成 csv文件
import csv

header = ['Number', 'text']
with open('../data/test_set.csv', 'w', encoding='utf-8', newline='') as f: # 解决空行的问题
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(results)

f.close()       # 记得关闭文件

In [6]:
# Count Vectors + RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv("../data/train_set.csv",encoding='ANSI')
test_df = pd.read_csv("../data/test_set.csv", encoding="utf-8")

vectorizer = CountVectorizer(max_features=3000)
# 定位 训练列
train_test = vectorizer.fit_transform(train_df['text'].values.astype('U'))
test_test = vectorizer.fit_transform(test_df['text'].values.astype('U'))

clf = RidgeClassifier()
# 这里可以设置 训练集的范文围 如 train_test[:10000] 训练集的 70%训练 30%预测
clf.fit(train_test, train_df['label'].values)

# 得到模型的得分
val_pred = clf.predict(train_test[10000:])

# 输出得分
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

# 测试集 预测
val_pred2 = clf.predict(test_test)

# 写入预测值至xlsx
filePath = "../data/type.xlsx"
wb = openpyxl.load_workbook(filePath)   # 打开excel文件

ws = wb['类别']

for i in range(2, ws.max_row+1):
    ws.cell(row=i, column=2, value = str(val_pred2[i-2]))

wb.save("../data/type.xlsx")

0.9201683620360309


In [ ]:
# 打包
# pyinstaller -p D:\\code\\Python_work\\NewSortEnd\\code -F test2.py --hidden-import sklearn.utils._weight_vector